# Yelp Core

In [1]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Additional Imports
import os,json,math,time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Open with secret file

In [2]:
import json
with open('/Users/rowenahan/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login["api-key"],timeout_s=5.0)
yelp_api

## Search for Favorite food
    - was orginal pizza but accdently deleted notebook
        -remade but kept getting error with n_results so switch to sushi

In [4]:
LOCATION = "Oahu, Hawaii"
TERM = "Sushi"

In [5]:
#Specifying JSON_FILE filename (can include a folder
#includ the search term in the filename)
JSON_FILE = "Data/results_in_progress_HNL_pizza.json"
JSON_FILE

'Data/results_in_progress_HNL_pizza.json'

## Making files 

In [6]:
# Check if JSON_FILE exists
file_exists=os.path.isfile(JSON_FILE)
#If it does not exist:
if file_exists==False:
    #Create any needed folders
    #get the folder Name only
    folder=os.path.dirname(JSON_FILE)
    #IF JSON_FILE included a folder
    if len(folder)>0:
        #create the folder
        os.makedirs(folder,exist_ok=True)

    #INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file")

    #save an empty list
    with open(JSON_FILE, "w") as f:
        json.dump([],f)
#if it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_HNL_pizza.json already exists.


In [7]:
#Load previous results and use len of results for offset
with open(JSON_FILE,"r") as f:
    previous_results = json.load(f)
#Set offset based on previous results
n_results = len(previous_results)
print(f"- {n_results} previous resuts found.")

- 0 previous resuts found.


In [8]:
search_results = yelp_api.search_query(location = LOCATION,
                                      term= TERM,
                                      offset=n_results)

search_results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
search_results["total"]
print(type(search_results))

<class 'dict'>


In [10]:
#How many did we get the details for?
results_per_page = len(search_results["businesses"])
results_per_page

20

In [11]:
#import additional packages for controlling our loop
import time, math
#Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((search_results["total"]-n_results)/results_per_page)
n_pages

47

In [12]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


## Loading files

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    #adds 200 ms pause
    time.sleep(.2)

  0%|          | 0/47 [00:00<?, ?it/s]

## Delete after testing and rerun

In [14]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [15]:
def create_json_file(JSON_FILE, delete_if_exists=False):
        #check if JSON_FILE exists
        file_exists=os.path.isfile(JSON_FILE)
        # if it DOSE exist:
        if file_exists == True:
            #check if user wants to delete if exists
            if delete_if_exists==True:
                print(f"[!]  {JSON_FILE} already exists. Deleting previous file...")
                #delet file and confirm it no longer exits
                os.remove(JSON_FILE)
                # recursive call to function after old file deleted
                create_json_file(JSON_FILE, delete_if_exists=False)
            else:
                print(f"[i] {JSON_FILE} already exists.")
        #if it dose NOT exist:
        else:
            #infor user and save empty list
            print(f"[i] {JSON_FILE} not found. Sving empty list to new file.")
            # Create any needed folders
            #Get the folder name only
            folder=os.path.dirname(JSON_FILE)
            #if JSON_FILE included a folder
            if len(folder)>0:
                #create the folder
                os.makedirs(folder,exist_ok=True)
            #save empty list to start the json file
            with open(JSON_FILE,"w") as f:
                json.dump([],f)

In [16]:
# Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE,delete_if_exists=True)
# loadprevious results and use en of results for offset
with open(JSON_FILE,"r") as f:
    previous_results=json.load(f)
#set offset based on previous results
n_results=len(previous_results)
print(f"- {n_results} previoous results found.")
#use our yelp_api variable"s search_query method to perform our API call
results=yelp_api.search_query(location=LOCATION,
                             term=TERM,
                             offset=n_results)
#how many results total?
total_results = results["total"]
#how many did we get the details for?
resutls_per_page = len(results["businesses"])
#use math ceil to round up for the total number of pages of results
n_pages = math.ceil((results["total"]-n_results)/results_per_page)
n_pages

[i] Data/results_in_progress_HNL_pizza.json not found. Sving empty list to new file.
- 0 previoous results found.


47

In [17]:
for i in tqdm_notebook(range(1,n_pages+1)):
    #Read in results in progress file and check the length
    with open(JSON_FILE,"r") as f:
        previous_results= json.load(f)
     #same number of results for the use as offset
    n_results=len(previous_results)
    if (n_results + results_per_page)>1000:
        print("Exceeded 1000 api calls. Stopping loop.")
        break

    #use n_results as the offset
    results=yelp_api.search_query(location=LOCATION,
                                 term=TERM,
                                 offset=n_results)
    #append new results and save to file
    previous_results.extend(results["businesses"])
    #display (previous_results)
    with open(JSON_FILE,"w") as f:
        json.dump(previous_results,f)

    time.sleep(.2)

  0%|          | 0/47 [00:00<?, ?it/s]

In [18]:
#load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,FxLPg0wJb-q76AFMd9oSHQ,the-clubhouse-mililani,The Clubhouse,https://s3-media2.fl.yelpcdn.com/bphoto/ocVYJL...,False,https://www.yelp.com/biz/the-clubhouse-mililan...,98,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 21.4540805, 'longitude': -158.022...",[],$$,"{'address1': '95-176 Kuahelani Ave', 'address2...",+18087840048,(808) 784-0048,6942.379386
1,MocT7ajwthZ7QAdz17xMPg,sushi-spot-aiea,Sushi Spot,https://s3-media1.fl.yelpcdn.com/bphoto/NUPtVy...,False,https://www.yelp.com/biz/sushi-spot-aiea?adjus...,1039,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 21.37878, 'longitude': -157.93136}",[delivery],$$,"{'address1': '99-209 Moanalua Rd', 'address2':...",+18084852255,(808) 485-2255,12122.550519
2,2V3peIO6IvumIPLoOie-hA,california-beach-rock-n-sushi-aiea-6,California Beach Rock n' Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/Fc9bG-...,False,https://www.yelp.com/biz/california-beach-rock...,346,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 21.381708, 'longitude': -157.93995}",[delivery],$$,"{'address1': '98-151 Pali Momi St', 'address2'...",+18085978000,(808) 597-8000,11564.444122
3,WSc-f2pM7I25HNn7kb0b9g,sushi-bay-kapolei,Sushi Bay,https://s3-media3.fl.yelpcdn.com/bphoto/JgzNV-...,False,https://www.yelp.com/biz/sushi-bay-kapolei?adj...,1800,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.0,"{'latitude': 21.3385086, 'longitude': -158.079...",[delivery],$$,"{'address1': '91-590 Farrington Hwy', 'address...",+18086939922,(808) 693-9922,20050.213924
4,QoxHbYIYxRh2OJmv6kbtlQ,kizuna-japanese-sushi-and-grill-kaneohe,Kizuna Japanese Sushi And Grill,https://s3-media2.fl.yelpcdn.com/bphoto/acFFA4...,False,https://www.yelp.com/biz/kizuna-japanese-sushi...,371,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 21.41379862638154, 'longitude': -...",[delivery],$$,"{'address1': '45-934 Kamehameha Hwy', 'address...",+18082341688,(808) 234-1688,18617.186416


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
928,MrN9jY6TMxxaeVqO-16_GA,glazers-coffee-honolulu,Glazer's Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/43knUv...,False,https://www.yelp.com/biz/glazers-coffee-honolu...,513,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"{'latitude': 21.291274, 'longitude': -157.820459}",[delivery],$$,"{'address1': '2700 S King St', 'address2': '',...",,,26019.673686
929,SmuR7oFRQ2EUuEAjNmeQYw,la-pizza-rina-honolulu,La Pizza Rina,https://s3-media3.fl.yelpcdn.com/bphoto/_siWGW...,False,https://www.yelp.com/biz/la-pizza-rina-honolul...,376,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 21.2980578, 'longitude': -157.839...",[delivery],$$,"{'address1': '1425 S King St', 'address2': '',...",+18089416634,(808) 941-6634,24294.771695
930,ExpKHaV8oY_FLrmjf3iDig,sharetea-honolulu,Sharetea,https://s3-media3.fl.yelpcdn.com/bphoto/GHMEmt...,False,https://www.yelp.com/biz/sharetea-honolulu?adj...,148,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",3.5,"{'latitude': 21.275531, 'longitude': -157.78593}",[delivery],$$,"{'address1': '4618 Kilauea Ave', 'address2': '...",+18087391368,(808) 739-1368,29562.680255
931,MiALjjd_s_Q-lSwUdjaFkQ,il-lupino-trattoria-and-wine-bar-honolulu,Il Lupino Trattoria & Wine Bar,https://s3-media1.fl.yelpcdn.com/bphoto/_xncz_...,False,https://www.yelp.com/biz/il-lupino-trattoria-a...,1047,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.0,"{'latitude': 21.278427202060843, 'longitude': ...","[pickup, delivery]",$$$,"{'address1': '2233 Kalakaua Ave', 'address2': ...",+18089223400,(808) 922-3400,26716.909566
932,kkeb2W1dwRIt_1-_5v4O8A,jamba-honolulu-21,Jamba,https://s3-media3.fl.yelpcdn.com/bphoto/HPpyr6...,False,https://www.yelp.com/biz/jamba-honolulu-21?adj...,186,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",3.0,"{'latitude': 21.290836856803516, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '1450 Ala Moana Blvd', 'address2'...",+18089416132,(808) 941-6132,24758.369148


## Checking for duplicates

In [19]:
final_df.duplicated(subset="id").sum()

0

In [20]:
final_df=final_df.drop_duplicates(subset="id")
final_df.duplicated(subset="id").sum()

0

In [21]:
final_df.to_csv("Data/final_results_HNL_pizza.csv.gz",
               compression="gzip",index=False)